<a href="https://colab.research.google.com/github/cherryblackk/Fis1220_Tarea_Recuperativa/blob/main/Tarea_Recuperativa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import random
import os
from datetime import datetime, timedelta

def generar_archivos_meteoros():
    # la semilla define todo el conjunto de datos
    # mismo numero = mismos archivos siempre
    # funciona igual que las semillas de minecraft, es decir, definen una combinación númerica aleatoria pero es unica
    # la semilla 1 es distinta a la 2, es como un codigo de barras que los identifica
    semilla_actual = 122025
    random.seed(semilla_actual)

    # definimos la ventana de tiempo de 2 dias como pide la tarea
    # desde el 1 de diciembre 2025 a las 2am hasta el 3 de diciembre 2025 a las 2am
    tiempo_inicio = datetime(2025, 12, 1, 2, 0, 0)
    tiempo_final = datetime(2025, 12, 3, 2, 0, 0)

    # calculamos cuantos segundos hay en total en 2 dias
    # esto nos sirve para distribuir los eventos en el tiempo
    duracion_total_segundos = int((tiempo_final - tiempo_inicio).total_seconds())

    # generamos un numero aleatorio de archivos entre 500 y 999
    # como especifica la tarea, debe ser entre esos valores
    cantidad_eventos = random.randint(500, 999)
    print(f"Generando {cantidad_eventos} archivos de meteoros.")

    # creamos la carpeta donde se guardaran los archivos si no existe
    if not os.path.exists("archivos_meteoros"):
        os.makedirs("archivos_meteoros")

    # creamos una lista con todos los segundos disponibles en los 2 dias
    # luego elegimos aleatoriamente algunos segundos para los eventos
    # random.sample asegura que no haya segundos repetidos
    segundos_disponibles = list(range(duracion_total_segundos))
    instantes_seleccionados = random.sample(segundos_disponibles, cantidad_eventos)

    # ordenamos los segundos de menor a mayor para que los eventos
    # esten en orden cronologico como pide la tarea
    instantes_seleccionados.sort()

    # ahora creamos cada archivo individualmente
    # usamos enumerate para tener un contador i que empieza en 0
    for i, segundos in enumerate(instantes_seleccionados):
        # calculamos el momento exacto del evento sumando los segundos al inicio
        tiempo_evento = tiempo_inicio + timedelta(seconds=segundos)

        # generamos los datos del meteoro segun los rangos especificados
        # duracion entre 0.1 y 5.0 segundos con 1 decimal
        duracion = round(random.uniform(0.1, 5.0), 1)

        # altura sobre el horizonte entre 10.0 y 90.0 grados
        altura = round(random.uniform(10.0, 90.0), 1)

        # acimut (direccion en el horizonte) entre 0.0 y 359.0 grados
        acimut = round(random.uniform(0.0, 359.0), 1)

        # formateamos el numero del archivo para que tenga 3 digitos
        # 1 se convierte en 001, 2 en 002, etc hasta 999
        numero_archivo = f"{i+1:03d}"

        # creamos el nombre del archivo con el formato requerido
        nombre_archivo = f"archivos_meteoros/meteoro_{numero_archivo}.txt"

        # abrimos el archivo en modo escritura y guardamos los datos
        with open(nombre_archivo, 'w') as archivo:
            # formateamos la linea con todos los datos separados por comas
            # fecha en formato YYYY-MM-DD, hora en HH:MM:SS
            linea = f"{tiempo_evento.strftime('%Y-%m-%d')}, {tiempo_evento.strftime('%H:%M:%S')}, {duracion}, {altura}, {acimut}"
            archivo.write(linea)

    # mensaje final confirmando que todo salio bien
    print(f"Se crearon {cantidad_eventos} archivos.")
    print(f"Estos archivos fueron generados por la seed {semilla_actual}.")

if __name__ == "__main__":
    generar_archivos_meteoros()

Generando 921 archivos de meteoros.
Se crearon 921 archivos.
Estos archivos fueron generados por la seed 122025.
